# 🌈 Chapter 3D-4: Shaders (GLSL)

This is where the magic happens.

A **Shader** is a small C-like program that runs for *every single pixel* on your object.

- If you render a 1920x1080 screen, this code runs **2 Million times per frame**.
- And it does it instantly.

We write shaders in **GLSL** (OpenGL Shading Language).

## 1. Passing Uniforms (Time)

Let's make a color that changes over time.

In [ ]:
fragment_shader = '''
    #version 330
    
    uniform float time; // We send this from Python
    
    out vec4 f_color;
    
    void main() {
        // Math functions like sin() are built-in and SUPER fast
        float red = (sin(time) + 1.0) / 2.0; // Oscillate 0 to 1
        float blue = (cos(time) + 1.0) / 2.0;
        
        f_color = vec4(red, 0.0, blue, 1.0);
    }
'''

# IN PYTHON LOOP:
# prog['time'].value = pygame.time.get_ticks() / 1000.0

## 2. Gradients (Fragment Position)

We can use the position of the pixel (`gl_FragCoord`) to create gradients.

In [ ]:
fragment_shader = '''
    #version 330
    out vec4 f_color;
    in vec3 v_pos; // Position from Vertex Shader
    
    void main() {
        // Mix color based on Y height
        f_color = vec4(v_pos.xy, 0.5, 1.0);
    }
'''

## 🛠️ Challenge: The Plasma

Plasma effects are created by combining multiple Sine waves.

1.  Pass `time` and `resolution` (screen width/height) to the shader.
2.  Calculate the distance of the pixel from the center.
3.  Use `sin(distance * 10.0 + time)` to make expanding rings.

### 🚀 Full Runnable Shader Example

Run this cell to see a Pulse Effect in action:

In [ ]:
import pygame
import moderngl
import numpy as np

pygame.init()
screen = pygame.display.set_mode((800, 600), pygame.OPENGL | pygame.DOUBLEBUF)
ctx = moderngl.create_context()

# FULL SCREEN QUAD (2 Triangles covering the screen)
quad_vertices = np.array([
    -1.0,  1.0, 0.0,
    -1.0, -1.0, 0.0,
     1.0,  1.0, 0.0,
     1.0, -1.0, 0.0,
], dtype='f4')

prog = ctx.program(
    vertex_shader='''
        #version 330
        in vec3 in_vert;
        void main() {
            gl_Position = vec4(in_vert, 1.0);
        }
    ''',
    fragment_shader='''
        #version 330
        uniform float time;
        out vec4 f_color;
        void main() {
            float red = (sin(time) + 1.0) / 2.0;
            f_color = vec4(red, 0.0, 1.0 - red, 1.0);
        }
    '''
)

vbo = ctx.buffer(quad_vertices.tobytes())
vao = ctx.simple_vertex_array(prog, vbo, 'in_vert')

timer = pygame.time.Clock()
start_time = pygame.time.get_ticks()

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT: running = False
            
    # Send Time to Shader
    current_time = (pygame.time.get_ticks() - start_time) / 1000.0
    prog['time'].value = current_time
    
    ctx.clear()
    # Render as Triangle Strip (Connects the 4 points into a quad)
    vao.render(moderngl.TRIANGLE_STRIP)
    
    pygame.display.flip()
    timer.tick(60)

pygame.quit()